# Supervised Learning
## Decision Trees - Tree ensembles

Author: Bingchen Wang

Last Updated: 31 Oct, 2022

---
<nav>
    <a href="../../Machine%20Learning.ipynb">Machine Learning</a> |
    <a href="../Supervised Learning.ipynb">Supervised Learning</a> |
    <a href="../Decision Trees.ipynb">Decision Trees</a>
</nav>

---

In [67]:
%%html
<link rel='stylesheet' type='text/css' media='screen' href='../styles/custom.css'>
<!--For a better format, run this code.-->

## Contents

- Popular ensemble methods
    - [Bootstrap aggregation/Bagging](#BA)
        - [Bagging](#BA1)
        - [On average, each bagged tree makes use of 2/3 of the observations.](#BA2)
        - [Out-of-Bag Error Estimation](#BA3)
        - [Variable Importance Measures](#BA4)
    - [Random Forests](#RF)
        - [Problem with bagging (high correlation)](#RF1)
        - [The random forest process](#RF2)
    - [Boosting](#Boosting)
        - [Central idea: fit tree to residuals](#Boosting1)
        - [Tunning parameters of boosting: $B$, $\lambda$ and $d$](#Boosting2)
        - [Boosting Algorithm for Regression Trees](#Boosting3)
        - [XGBoost](https://xgboost.readthedocs.io/en/stable/tutorials/model.html)
    - [Bayesian additive regression trees](#BART)
        - [Bayesian additive regression trees algorithm](#BART1)
- Implementation

<a name = "BA"></a>
## Bootstrap aggregation/Bagging

<blockquote>
    Bootstrap aggregation, or <b>bagging</b>, is a general-purpose procedure for <b>reducing the variance</b> of a statistical learning method. -- Introduction to Statistical Learning, p.340. 
</blockquote>

<div class = "alert alert-block alert-info">
    <b>Central idea: Averaging a set of imperfectly correlated observations reduces variance.</b><br>
    Consider a set of $n$ <em>independent</em> observations $Z_1, \dots, Z_n$, each with variance $\sigma^2$. Then the variance of the mean $\bar Z$ of the observations is given by $\sigma^2/n$.
</div>

<a name = "BA1"></a>
### Bagging
Given a training data set, create $B$ bootstrapped training data sets by taking repeated samples (with replacement) from the original data set. For each of the bootstrapped data set ($b = 1, \dots, B$), train the method to get prediction rule $\hat f^{*b}(x)$. Finally, average all the predictions to get:
$$
\hat f_{\text{bag}}(x) = \frac{1}{B}\sum_{b=1}^B \hat{f}^{*b}(x).
$$

In the context of decision trees, each decision tree trained on a boostrapped data set is grown deep and *not pruned*, thereby having low bias and high variance.

<a name = "BA2"></a>
### On average, each bagged tree makes use of 2/3 of the observations.

Suppose that the size of training data set is $n$ (which is also the size of the bootstrapped data set), the chance of an observation not being selected by a bootstrapped data set is:
$$
(1-\frac{1}{n})^n
$$
<div style = "text-align: center;">
    <img src="./convergence.png" style="width:50%;" > <br>
    Note how quickly it converges, with $n = 5$ very close to $1/3$. 
</div>
<br>

**Claim**: $\lim_{n\rightarrow \infty}(1-\frac{1}{n})^n = 1/e$ <br>
**Proof**:
$$
\begin{align}
\lim_{n\rightarrow \infty}(1-\frac{1}{n})^n = & \lim_{n\rightarrow \infty}(1-\frac{1}{n})^n \\
= & \lim_{n\rightarrow \infty} \frac{1}{(\frac{n}{n-1})^n} \\
= & \lim_{n\rightarrow \infty} \frac{1}{(1 + \frac{1}{n-1})(1 + \frac{1}{n-1})^{(n-1)}} \\
= & \frac{1}{\underbrace{\lim_{n\rightarrow \infty} (1 + \frac{1}{n-1})}_{1}\underbrace{\lim_{n\rightarrow \infty} (1 + \frac{1}{n-1})^{(n-1)}}_{e}} \\
= & \frac{1}{e}
\end{align}
$$
<details>
    <summary><font size="3"><b>Additional proof ($\lim_{n\rightarrow \infty}(1+\frac{1}{n})^n = e$) <br></b></font></summary>
Consider $x$ in the interval $[1, 1 + \frac{1}{n}]$,
$$
\begin{align}
\frac{1}{1 + \frac{1}{n}} \leq \frac{1}{x} \leq 1 \\
\int_{1}^{1 + \frac{1}{n}} \frac{1}{1 + \frac{1}{n}} dx \leq  \int_{1}^{1 + \frac{1}{n}}\frac{1}{x} dx \leq \int_{1}^{1 + \frac{1}{n}} 1 dx \\
\frac{1}{n+1} \leq \ln \left(1 + \frac{1}{n}\right) \leq \frac{1}{n} \\
e^{\frac{1}{n+1}} \leq 1 + \frac{1}{n} \leq e^{\frac{1}{n}}
\end{align}
$$
Then, focus on the left inequality:
$$
\begin{align}
e \leq & {\left(1 + \frac{1}{n}\right)}^{n+1} \\
\frac{e}{1 + \frac{1}{n}} \leq & {\left(1 + \frac{1}{n}\right)}^{n}
\end{align}
$$
Thus, $\lim_{n\rightarrow \infty}{\left(1 + \frac{1}{n}\right)}^{n} \geq e$. <br>
Next, focus on the right inequality:
$$
{\left(1 + \frac{1}{n}\right)}^{n} \leq e
$$
Thus, $\lim_{n\rightarrow \infty}{\left(1 + \frac{1}{n}\right)}^{n} \leq e$. <br>
Therefore, $\lim_{n\rightarrow \infty}{\left(1 + \frac{1}{n}\right)}^{n} = e$. <br>    
Detailed proof: <a href = "http://aleph0.clarku.edu/~djoyce/ma122/elimit.pdf">here</a>
</details>

<a name = "BA3"></a>
### Out-of-Bag (OOB) Error Estimation
<blockquote>
    One can show that on average, each bagged tree makes use of around two-thirds of the observations. The remaining one-third of the observations not used to fit a given bagged tree can are referred to as the <b>out-of-bag</b> (OOB) observations. -- Introduction to Statistical Learning, p.342. <br> <br>
    The OOB approach for estimating the test error is particularly convenient when performing bagging on large data sets for which cross-validation would be computationally onerous. -- Introduction to Statistical Learning, p.343.
</blockquote>
This provides a way to estimate the test error without the need to perform cross-validation or the validation set approach.

#### The Process:
1. For each observation $x_i$, there will be around $B/3$ predictions (using trees in which $x_i$ is OOB).
2. Obtain a single prediction (call it the OOB prediction) through averaging (regression) or majority vote (classification).
3. Calculate the overall OOB MSE (regression) or classification error (classification).

<div class ="alert alert-block alert-warning"> With $B$ sufficiently large, it can be shown that the OOB error is virtually equivalent to the leave-one-out cross-validation error.</div>

<a name = "BA4"></a>
### Variable Importance Measures
- decrease in the Residual Sum of Squares (RSS) (for bagging regression trees)
    - add up the **total** amount of decrease in the RSS due to splits over a given predictor
    - **average** over all $B$ trees
- decrease in the Gini index (for bagging classification trees)
    - add up the **total** amount of decrease in the Gini index/cross-entropy due to splits over a given predictor
    - **average** over all $B$ trees
    
<a name = "RF"></a>    
## Random Forests

<a name = "RF1"></a> 
### Problem with bagging
**Trees can look fairly similar** in the presence of strong dominant predictors, resulting in highly correlated predictions. Averaging highly correlated quantities does not lead to a large reduction in variance.

Random forests improves upon bagged trees by **decorrelating** the trees through incorporating additional randomness in the splitting process.

<a name = "RF2"></a> 
### The random forest process
1. Get $B$ bootstrapped training data sets.
2. Train a decision tree on each bootstrapped data set. Each time a split in a tree is considered, a random sample of $m \approx \sqrt{p}$ predictors is considered rather than the full predictors set. A fresh sample of $m$ predictors is taken at **each split** within a decision tree.

<div class ="alert alert-block alert-info"> When $m = p$, a random forest amounts simply to bagging. Using a small value of $m$ will typically be helpful in the presence of a large number of highly correlated predictors (e.g., genomics and biology).</div>

<a name = "Boosting"></a>
## Boosting

<blockquote>
    Like bagging, <b>boosting</b>, is a general approach that can be applied to many statistical learning methods for regression and classification. -- Introduction to Statistical Learning, p.345. 
</blockquote>

In boosting, trees are grown **sequentially**, with information from previously grown trees feeding into subsequent trees. Boosting does not use bootstrapping but **modified versions** of the original data set.

<a name = "Boosting1"></a>
### Central idea
Fit a decision tree to the **residuals** from the previous model.

<a name = "Boosting2"></a>
### Tunning parameters of boosting:
1. **The number of trees $B$.** Unlike bagging and random forests, boosting can *overfit* if $B$ is too large, although this overfitting tends to occur slowly if at all. We use cross-validation to select $B$.
2. **The shrinkage parameter $\lambda$ (a small positive number).** This controls *the rate at which boosting learns*. Typical values are 0.01 or 0.001, and the right choice can depend on the problem. To achieve good performance, a small $\lambda$ may require a large $B$.
3. **The number $d$ of splits in each tree,** which governs the *complexity* of the boosted ensemble. $d$ splits lead to $d+1$ terminal nodes. Often $d=1$ works well, in which ease each tree is a *stump*, consisting of a single split.

<div style = "text-align: center;">
    <img src="../images/slumps vs depth-2 decision tree.pdf" style="width:80%;" > <br>
    Illustration: a single decision tree with 2 splits vs average of two slumps
</div>

<a name = "Boosting3"></a>
### Boosting Algorithm for Regression Trees

<section class = "section--algorithm">
    <div class = "algorithm--header"> Boosting for Regression Trees</div>
    <div class = "algorithm--content">
        Set $\hat f(x) = 0$ and $r_i = y_i$ for all $i$ in the training set. <br>
        For $b = 1, \dots, B$, repeat:
        <blockquote>
            <ol>
                <li> Fit a tree $\hat f^b$ with $d$ splits ($d+1$ terminal nodes) to the training data $(X, r)$.
                <li> Update $\hat f$ by adding in a shrunken version of the new tree:
                    $$
                    \hat f(x) \leftarrow \hat f(x) + \lambda \hat f^b(x)
                    $$
                <li> Update the residuals,
                    $$
                    r_i \leftarrow r_i - \lambda \hat f^b(x_i)
                    $$
            </ol>
        </blockquote>
        Output the boosted model,
        $$
        \hat f(x) = \sum_{b=1}^B \lambda \hat f^b(x).
        $$
    </div>
</section>

### XGBoost
A nice tutorial: <a href = "https://xgboost.readthedocs.io/en/stable/tutorials/model.html">here</a>.

<a name = "BART"></a>
## Bayesian additive regression trees (BART)
<blockquote>
    BART is related to both approaches (bagging/RF and boosting): each tree is constructed in a random manner as in bagging and random forests, and each tree tries to capture signal not yet accounted for by the current model, as in boosting. -- Introduction to Statistical Learning, p.348. <br><br>
    It turns out that the BART method can be viewed as a <em>Bayesian</em> approach to fitting an ensemble of tree: each time we randomly perturb a tree in order to fit the residuals, we are in fact drawing a new tree from the posterior distribution. -- Introduction to Statistical Learning, p.350.
</blockquote>

#### New notation
- $K$: the number of regression trees
- $B$: the number of iterations for which the BART algrithm will be run
- $L$: the number of burn-in iterations
- $\hat f_k^b(x)$: the prediction at $x$ for the $k$th regression tree used in the $b$th iteration.
- $\hat f^b(x) = \sum_{k=1}^K \hat f_k^b(x)$: the prediction at $x$ in the $b$th iteration.

#### Partial residual
To update the $k$th tree in the $b$th iteration, construct a **partial residual**:
$$
r_i = y_i -\sum_{k^\prime < k} \hat f_{k^\prime}^b(x_i) - \sum_{k^\prime > k} \hat f_{k^\prime}^{b-1}(x_i)
$$
for $i = 1, \dots, n$.
#### Random perturbation
Rather than fitting a fresh tree based on the partial residual, BART **randomly chooses a perturbation** to $f_k^{b-1}$ from a set of possible perturbations, favouring ones that *improve the fit to the partial residual*:
1. Change the structure of the tree by adding or prunning branches.
2. Change the prediction in each terminal node of the tree.

<div class = "alert alert-block alert-info">Roughly speaking, this <em>guards against overfitting</em> as it limits how "hard" we fit the data in each iteration.</div>

<div style = "text-align: center;">
    <img src="../images/Random perturbations.jpeg" style="width:70%;" > <br>
    Source: Introduction to Statistical Learning, p.349.
</div>

#### Burn-in period
Models obtained in the earlier iterations–known as the burn-in period–tend not to provide very good results. Thus, we usually discard the first few prediction models $b = 1, \dots, L$.

#### Model prediction and uncertainty
$$
\hat f(x) = \frac{1}{B-L} \sum_{b = B-L}^B \hat f^b(x)
$$
The percentiles of $\hat f^{L+1}(x), \dots, \hat f^B(x)$ provide a measure of uncertainty in the final prediction.

<a name = "BART1"></a>
### Bayesian Additive Regression Trees

<section class = "section--algorithm">
    <div class = "algorithm--header"> Bayesian Additive Regression Trees </div>
    <div class = "algorithm--content">
        Let $\hat f^1_1(x) = \hat f^1_2(x) = \cdots = \hat f^1_K(x) = \frac{1}{nK} \sum_{i=1}^n y_i$ and thus 
        $$
        \hat f^1(x) = \sum_{k=1}^K \hat f^1_k(x) = \frac{1}{n}\sum_{i=1}^n y_i \; \textbf{(sample mean)}
        $$
        For $b = 2, \dots, B$:
        <blockquote>
            <ol>
                <li> For $k = 1, \dots, K$:
                    <ol>
                        <li> Compute the current partial residual:
                            $$
                            r_i = y_i -\sum_{k^\prime < k} \hat f_{k^\prime}^b(x_i) - \sum_{k^\prime > k} \hat f_{k^\prime}^{b-1}(x_i)
                            $$
                            for $i = 1, \dots, n$.
                        <li> Fit a new tree, $\hat f^b_k(x)$ to $\{r_i\}$ by randomly perturbaing the $k$th tree from the previous iteration $f_k^{b-1}$, favuring the ones that improve the fit.
                    </ol>
                <li> Compute $\hat f^b(x) = \sum_{k=1}^K \hat f_k^b(x)$.
            </ol>
        </blockquote>
        Output the BART model,
        $$
        \hat f(x) = \frac{1}{B-L} \sum_{b=B-L}^B \hat f^b(x).
        $$
    </div>
</section>
<blockquote>
    BART has been shown to have very impressive out-of-box performance–that is, it performs very well with minimal tunning. -- Introduction to Statistical Learning, p.351.
</blockquote>
